In [1]:
from sklearn.datasets import fetch_mldata
import pandas as pd
import numpy as np

In [2]:
# 读取数据
mnist = fetch_mldata('MNIST original',data_home='./datasets/')
# 得到数据X和标签y
X, y = mnist['data'], mnist['target']
# 标签的副本拷贝
y_orig = y.copy()

/usr/local/anaconda3/envs/python35/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/anaconda3/envs/python35/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
# 找出所有label为1的正样本的索引
select_index = np.array(np.where(y==1)).reshape(-1).tolist()
# 随机选择size大小的index作为positive数据
select_index_size = np.random.choice(select_index,replace=False, size=3000)
# 其他的标签都改成0，认为是unlabelled
other_index = [ i for i in range(70000) if i not in select_index_size]
y[other_index] = 0

In [4]:
from sklearn.ensemble import RandomForestClassifier

In [5]:
rf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)

In [6]:
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [7]:
results = pd.DataFrame({
    'truth'      : y_orig,                    # True labels
    'label'      : y,                         # Labels shown to models
    'output_std' : rf.predict_proba(X)[:,1]   # Random forest's scores
}, columns = ['truth', 'label', 'output_std'])

In [36]:
#模型输出的正样本为A，真正的正样本集合为B
# 计算精确率（Precision）,指的是模型判别为正的所有样本中有多少是真正的正样本,则Precision（A,B）=|A∩B| / |A|
A_B = len(results[(results['output_std'] >= 0.08) & (results['truth'] == 1)])
A = len(results[results['output_std'] >= 0.08])
Precision_A_B = A_B / A * 100
print("直接应用标准分类器，计算精确率Precision = %.3f %%" % Precision_A_B)

直接应用标准分类器，计算精确率Precision = 99.545 %


In [37]:
# 计算召回率（Recall）,指的是所有正样本有多少被模型判定为正样本Recall（A,B） = |A∩B| / |B|
B = len(results[results['truth'] == 1])
Recall_A_B = A_B / B * 100
print("直接应用标准分类器，计算精确率Recall = %.3f %%" % Recall_A_B)

直接应用标准分类器，计算精确率Recall = 97.220 %
